# SCRAPING AND MODELLING AIRBNB IN NAIROBI

In this project, I scraped data on Airbnb Nairobi from Airbnb website and use regression modelling to predict prices.

In [37]:
#Importing the packages
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [6]:
url='https://www.airbnb.com/s/nairobi--kenya/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&date_picker_type=calendar&checkin=2022-10-13&checkout=2022-10-31&adults=3&source=structured_search_input_header&search_type=filter_change'
soup=BeautifulSoup(requests.get(url).content, 'html.parser')
print(soup.prettify())

soup.find_all('div', "c1l1h97y dir dir-ltr")

listings=soup.find_all('div', "c1l1h97y dir dir-ltr")

listings[2]

<!DOCTYPE html>
<html data-hyperloop-version="1" data-is-hyperloop="true" dir="ltr" lang="en">
 <meta charset="utf-8"/>
 <meta content="en" name="locale"/>
 <meta content="notranslate" name="google"/>
 <meta content="authenticity_token" id="csrf-param-meta-tag" name="csrf-param"/>
 <meta content="" id="csrf-token-meta-tag" name="csrf-token"/>
 <meta content="" id="english-canonical-url"/>
 <meta content="on" name="twitter:widgets:csp"/>
 <meta content="yes" name="mobile-web-app-capable"/>
 <meta content="yes" name="apple-mobile-web-app-capable"/>
 <meta content="Airbnb" name="application-name"/>
 <meta content="Airbnb" name="apple-mobile-web-app-title"/>
 <meta content="#ffffff" name="theme-color"/>
 <meta content="#ffffff" name="msapplication-navbutton-color"/>
 <meta content="black-translucent" name="apple-mobile-web-app-status-bar-style"/>
 <meta content="/?utm_source=homescreen" name="msapplication-starturl"/>
 <script>
  (function() {
  var pgRequest = new XMLHttpRequest();
  var 

<div class="c1l1h97y dir dir-ltr" style="--transition-element_transition-delay:100ms;--transition-element_transition-duration:200ms"><div itemprop="itemListElement" itemscope="" itemtype="http://schema.org/ListItem"><meta content="Modern 2BR Near Yaya Center-Breathtaking Views" itemprop="name"/><meta content="3" itemprop="position"/><meta content="www.airbnb.com/rooms/633890474501476852?adults=3&amp;check_in=2022-10-13&amp;check_out=2022-10-31&amp;previous_page_section_name=1000" itemprop="url"/><div><div><div aria-labelledby="title_633890474501476852" class="cy5jw6o dir dir-ltr" role="group"><a aria-labelledby="title_633890474501476852" class="ln2bl2p dir dir-ltr" href="/rooms/633890474501476852?adults=3&amp;check_in=2022-10-13&amp;check_out=2022-10-31&amp;previous_page_section_name=1000&amp;federated_search_id=085e9b52-cfb7-4481-b809-c4be66ea2a4f" rel="noopener noreferrer nofollow" target="listing_633890474501476852"></a><div class="mz543g6 dir dir-ltr"><div class="c14dgvke dir dir-l

#get the link

In [7]:
listings[1].find_all('a')[1].get('href')

listings[1].get_text()

'1 of 6 items showingSuperhostApartment in NairobiNairobi Hill Elegance- Upper Hill 2 bedrooms3 beds,  · 2 bedrooms$55\xa0$50\xa0night$50 per night, originally $55 · $906 total$906 totalShow price breakdown4.98 (102)'

In [8]:
def extract_basic_features(listing_html):
    features_dict ={}
    
    url=listing_html.find('a').get('href')
    name=listing_html.find("span", {"class":"t6mzqp7 dir dir-ltr"}).get_text()
    header=listing_html.find("div",{"class":"t1jojoys dir dir-ltr"}).get_text()
    
    features_dict['url']=url
    features_dict['name']=name
    features_dict['header']=header
    
    return features_dict

extract_basic_features(listings[6])

{'url': '/rooms/669551357712309910?adults=3&check_in=2022-10-13&check_out=2022-10-31&previous_page_section_name=1000&federated_search_id=085e9b52-cfb7-4481-b809-c4be66ea2a4f',
 'name': 'Executive 2 Bedroom Apartment for Rent in Kilimani',
 'header': 'Apartment in Kilimani Estate'}

In [9]:
RULES_SEARCH_PAGE = {
    'url': {'tag':'a', 'get': 'href'},
    'name':{'tag': 'span','class': 't6mzqp7 dir dir-ltr'},
    'header':{'tag': 'div', 'class': 't1jojoys dir dir-ltr'},
    'facilities':{'tag': 'div', 'class': 'f15liw5s s1cjsi4j dir dir-ltr'},
    'rooms':{'tag': 'div', 'class': 'f15liw5s s1cjsi4j dir dir-ltr', 'order': 1},
    'ratings_n_reviews':{'tag' : 'span', 'class': 'ru0q88m dir dir-ltr'},
    'price':{'tag' : 'span', 'class': '_tyxjp1'},
}

In [10]:
#extract the right tag
def extract_element(listing_html,params):
    if 'class' in params:
        elements_found=listing_html.find_all(params['tag'],params['class'])
    else:
        elements_found=listing_html.find_all(params['tag'])
#extract the right element
    tag_order=params.get('order',0)
    element=elements_found[tag_order]

#get text
    if 'get' in params:
        output=element.get(params['get'])
    else:
        output=element.get_text()
    return output
    

In [11]:
print(extract_element(listings[0],RULES_SEARCH_PAGE['price']))
print(extract_element(listings[0],RULES_SEARCH_PAGE['url']))
print(extract_element(listings[0],RULES_SEARCH_PAGE['name']))
print(extract_element(listings[0],RULES_SEARCH_PAGE['header']))
print(extract_element(listings[0],RULES_SEARCH_PAGE['ratings_n_reviews']))
print(extract_element(listings[0],RULES_SEARCH_PAGE['facilities']))
print(extract_element(listings[0],RULES_SEARCH_PAGE['rooms']))

$32
/rooms/28650254?adults=3&check_in=2022-10-13&check_out=2022-10-31&previous_page_section_name=1000&federated_search_id=085e9b52-cfb7-4481-b809-c4be66ea2a4f
9th Floor View
Apartment in Nairobi
4.86 (23)

3 beds,  · 2 bedrooms


In [12]:
for feature in RULES_SEARCH_PAGE:
    print(f"{feature}: {extract_element(listings[19],RULES_SEARCH_PAGE[feature])}")

url: /rooms/13524006?adults=3&check_in=2022-10-13&check_out=2022-10-31&previous_page_section_name=1000&federated_search_id=085e9b52-cfb7-4481-b809-c4be66ea2a4f
name: Gorgeous Getaway Near Yaya Center
header: Apartment in Kileleshwa
facilities: 
rooms: 4 beds,  · 3 bedrooms
ratings_n_reviews: 4.79 (152)
price: $57


In [13]:
url

'https://www.airbnb.com/s/nairobi--kenya/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&date_picker_type=calendar&checkin=2022-10-13&checkout=2022-10-31&adults=3&source=structured_search_input_header&search_type=filter_change'

In [14]:
def get_listings(search_page):
    soup=BeautifulSoup(requests.get(search_page).content,'html.parser')
    listings=soup.find_all('div','c1l1h97y dir dir-ltr')
    return listings

In [15]:
len(get_listings(url))

new_url=url+'&items_offset=20'
len(get_listings(new_url))

20

In [16]:
print(extract_element(get_listings(url)[0],RULES_SEARCH_PAGE['name']))
print(extract_element(get_listings(new_url)[0],RULES_SEARCH_PAGE['name']))

9th Floor View
A contemporary and homely 3 BR Apt near yaya ctr.


In [17]:
import time
all_listings=[]
for i in range(15):
    offset=20*i
    new_url=url+f'&items_offset={offset}'
    new_listings=get_listings(new_url)
    all_listings.extend(new_listings)
    print(len(all_listings))
    
    time.sleep(2)

20
40
60
80
100
120
140
160
180
200
220
240
260
280
300


In [18]:
print(extract_element(all_listings[98],RULES_SEARCH_PAGE['name']))

Shilz Suites


In [19]:
def build_urls(main_url,listings_per_page=20,pages_per_location=15):
    url_list=[]
    for i in range(pages_per_location):
        offset=listings_per_page*i
        url_pagination=main_url+f'&items_offset={offset}'
        url_list.append(url_pagination)
    return url_list

In [20]:
def extract_page_features(soup,rules):
    features_dict={}
    for feature in rules:
        try:
            features_dict[feature]=extract_element(soup,rules[feature])
        except:
            features_dict[feature]='empty'
    return features_dict

In [24]:
#scrape pages iteratively
def process_search_pages(url_list):
    features_list=[]
    for page in url_list:
        listings=get_listings(page)
        for listing in listings:
            features=extract_page_features(listing,RULES_SEARCH_PAGE)
            features_list.append(features)
    return features_list

In [25]:
url_list=build_urls(url)
url_list

['https://www.airbnb.com/s/nairobi--kenya/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&date_picker_type=calendar&checkin=2022-10-13&checkout=2022-10-31&adults=3&source=structured_search_input_header&search_type=filter_change&items_offset=0',
 'https://www.airbnb.com/s/nairobi--kenya/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&date_picker_type=calendar&checkin=2022-10-13&checkout=2022-10-31&adults=3&source=structured_search_input_header&search_type=filter_change&items_offset=20',
 'https://www.airbnb.com/s/nairobi--kenya/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&date_picker_type=calendar&checkin=2022-10-13&checkout=2022-10-31&adults=3&source=structured_search_input_header&search_type=filter_change&items_offset=40',
 'https://www.airbnb.com/s/nairobi--kenya/homes?tab_id=h

In [42]:
base_features = process_search_pages(url_list[0:15])

In [43]:
vs=pd.DataFrame(base_features)
vs

,url,name,header,facilities,rooms,ratings_n_reviews,price
0,/rooms/28650254?adults=3&check_in=2022-10-13&c...,9th Floor View,Apartment in Nairobi,,"3 beds, · 2 bedrooms",4.86 (23),$32
1,/rooms/20040341?adults=3&check_in=2022-10-13&c...,Nairobi Hill Elegance- Upper Hill 2 bedrooms,Apartment in Nairobi,,"3 beds, · 2 bedrooms",4.98 (102),$50
2,/rooms/633890474501476852?adults=3&check_in=20...,Modern 2BR Near Yaya Center-Breathtaking Views,Apartment in Nairobi,,"2 beds, · 2 bedrooms",5.0 (6),$54
3,/rooms/43466304?adults=3&check_in=2022-10-13&c...,Kilimani Luxe Apartment steps from Yaya Centre,Condo in Kileleshwa,,"3 beds, · 3 bedrooms",5.0 (3),$49
4,/rooms/46253912?adults=3&check_in=2022-10-13&c...,"Relaxing, Warm, Airy & Homely Apt close to malls",Apartment in Nairobi,,"2 beds, · 2 bedrooms",4.84 (19),$53
...,...,...,...,...,...,...,...
295,/rooms/51823641?adults=3&check_in=2022-10-13&c...,Fergusson house is a great stop and layover,Apartment in Parklands,,"2 beds, · 2 bedrooms",4.8 (5),$51
296,/rooms/662038183458169860?adults=3&check_in=20...,African Beauty,Condo in Nairobi,,"4 beds, · 3 bedrooms",4.83 (6),$92
297,/rooms/52878675?adults=3&check_in=2022-10-13&c...,Palatial view- 2br MASTER Ensuite,Apartment in Kileleshwa,,"4 beds, · 2 bedrooms",New,$53
298,/rooms/10912864?adults=3&check_in=2022-10-13&c...,An Irish welcome in Karen - Hill Cottage,Home in Nairobi,,"2 beds, · 2 bedrooms",4.73 (107),$67


In [57]:
Excel=pd.ExcelWriter("Airbnb.xlsx")

vs.to_excel(Excel)

Excel.save()